In [1]:
# Importamos librerías necesarias

import requests
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
# URL base de la API de CoinCap
base_url = "https://api.coincap.io/v2"


In [3]:
# Endpoint para obtener datos de activos
endpoint = "/assets"

In [4]:
# Encabezados de la solicitud con la clave API
headers = {
    "Accept-Encoding": "gzip, deflate"}

In [5]:
# URL completa para la solicitud
url = base_url + endpoint

In [6]:
# Realizar la solicitud GET a la API
response = requests.get(url, headers=headers)

In [7]:
# Verificar el código de estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa
    data = response.json()
    # Imprimir los datos de los activos (solo imprime los primeros 5 en este ejemplo)
    for asset in data['data'][:5]:
        print(asset)
else:
    # La solicitud falló
    print("Error al realizar la solicitud:", response.status_code)


{'id': 'bitcoin', 'rank': '1', 'symbol': 'BTC', 'name': 'Bitcoin', 'supply': '19685775.0000000000000000', 'maxSupply': '21000000.0000000000000000', 'marketCapUsd': '1235363300388.9287958480714225', 'volumeUsd24Hr': '7341729264.2379000069559735', 'priceUsd': '62754.1105386467535999', 'changePercent24Hr': '-1.8064028304167488', 'vwap24Hr': '62732.5386310577606449', 'explorer': 'https://blockchain.info/'}
{'id': 'ethereum', 'rank': '2', 'symbol': 'ETH', 'name': 'Ethereum', 'supply': '120072116.6801383000000000', 'maxSupply': None, 'marketCapUsd': '381013768280.9152805825175488', 'volumeUsd24Hr': '5815466411.4731633367792303', 'priceUsd': '3173.2077256196199002', 'changePercent24Hr': '-4.9680474296376156', 'vwap24Hr': '3204.8033951901477238', 'explorer': 'https://etherscan.io/'}
{'id': 'tether', 'rank': '3', 'symbol': 'USDT', 'name': 'Tether', 'supply': '109031963304.8744800000000000', 'maxSupply': None, 'marketCapUsd': '109109083895.0969952040657781', 'volumeUsd24Hr': '15978129201.3001364

In [8]:
# Convertir el JSON a un DataFrame de pandas
df = pd.DataFrame(data['data'])

In [9]:
#Agregamos columna fecha de consulta
fecha_consulta = datetime.now().timestamp()
df['fecha_consulta'] = pd.to_datetime(fecha_consulta, unit='s')

In [10]:
# Eliminar columnas innecesarias
df.drop(['id'], axis=1, inplace=True)

In [11]:
# Elegimos la columna symbol como ID. Reordenamos y renombramos
symbol_column = df.pop('symbol')  
df.insert(0, 'id', symbol_column)

In [12]:
# Creamos la conexión a Redsshift
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="matiaspereyra_coderhouse"
with open("C:/Users/machi/Desktop/Cursos/Data Engineer/Coder/pwdredshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [13]:
#Crear cursor
cursor = conn.cursor()

In [14]:
#Crear la tabla
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS matiaspereyra_coderhouse.crypto
        (
	    id VARCHAR(50) primary key  
	    ,rank INTEGER
	    ,name VARCHAR(255)  
	    ,supply DECIMAL
	    ,maxSupply DECIMAL
	    ,marketCapUsd DECIMAL
	    ,volumeUsd24Hr DECIMAL  
	    ,priceUsd DECIMAL 
	    ,changePercent24Hr DECIMAL  
	    ,vwap24Hr DECIMAL
	    ,explorer VARCHAR(255)
	    ,fecha_consulta TIMESTAMP
	    ,fecha_insercion TIMESTAMP
	    )
    """)
    conn.commit()

In [15]:
# Eliminar los registros existentes
cursor.execute("DELETE FROM crypto")
conn.commit()

In [18]:
df.head()

,id,rank,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer,fecha_consulta
0,BTC,1,Bitcoin,19685775.0000000000000000,21000000.0000000000000000,1235363300388.9287958480714225,7341729264.2379000069559735,62754.1105386467535999,-1.8064028304167488,62732.5386310577606449,https://blockchain.info/,2024-04-29 19:09:18.525556992
1,ETH,2,Ethereum,120072116.6801383000000000,None,381013768280.9152805825175488,5815466411.4731633367792303,3173.2077256196199002,-4.9680474296376156,3204.8033951901477238,https://etherscan.io/,2024-04-29 19:09:18.525556992
2,USDT,3,Tether,109031963304.8744800000000000,None,109109083895.0969952040657781,15978129201.3001364866670266,1.0007073209349342,0.0186013617483344,1.0000092703844529,https://www.omniexplorer.info/asset/31,2024-04-29 19:09:18.525556992
3,BNB,4,BNB,166801148.0000000000000000,166801148.0000000000000000,98568026659.4718950500250056,358035230.0364964918872859,590.9313445460932622,-2.4684278366642165,591.2645697608723939,https://etherscan.io/token/0xB8c77482e45F1F44d...,2024-04-29 19:09:18.525556992
4,SOL,5,Solana,446702697.3313378700000000,None,60214373004.1416565138681220,716976255.6766622177428654,134.7974242463061849,-5.9330164183301314,136.6830518799166738,https://explorer.solana.com/,2024-04-29 19:09:18.525556992


In [19]:
# Insertamos registros
for index, row in df.iterrows():
    cursor.execute("INSERT INTO crypto (id, rank, name, supply, maxSupply, marketCapUsd, volumeUsd24Hr, priceUsd, changePercent24Hr, vwap24Hr, explorer, fecha_consulta, fecha_insercion) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   (row['id'], row['rank'], row['name'], row['supply'], row['maxSupply'], row['marketCapUsd'], row['volumeUsd24Hr'], row['priceUsd'], row['changePercent24Hr'], row['vwap24Hr'], row['explorer'], row['fecha_consulta'], datetime.now()))


# Confirmar la inserción de los registros
conn.commit()



In [20]:
# Consulta
query = "SELECT * FROM crypto"

In [21]:
# Leer la tabla en un DataFrame
df_result = pd.read_sql_query(query, conn)

C:\Users\machi\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

In [23]:
# Mostrar el DataFrame resultante
print(df_result)

      id  rank                name        supply     maxsupply  marketcapusd  \
0    BTC     1             Bitcoin  1.968578e+07  2.100000e+07  1.235363e+12   
1    ETH     2            Ethereum  1.200721e+08           NaN  3.810138e+11   
2   USDT     3              Tether  1.090320e+11           NaN  1.091091e+11   
3    BNB     4                 BNB  1.668011e+08  1.668011e+08  9.856803e+10   
4    SOL     5              Solana  4.467027e+08           NaN  6.021437e+10   
..   ...   ...                 ...           ...           ...           ...   
95   TWT    96  Trust Wallet Token  4.166499e+08  1.000000e+09  4.285587e+08   
96   ELF    97                aelf  7.263902e+08  1.000000e+09  4.234347e+08   
97  QTUM    98                Qtum  1.047558e+08  1.078224e+08  4.227932e+08   
98   ZIL    99             Zilliqa  1.737220e+10  2.100000e+10  4.209497e+08   
99   HOT   100                Holo  1.732390e+11           NaN  4.182218e+08   

    volumeusd24hr  priceusd  changeperc